In [3]:
import csv

file = 'cleaned2.csv'

with open(file, 'r') as fi:
    reader = csv.reader(fi)
    headers = next(reader)
    for i in range(len(headers)):
        print(i, headers[i])     

0 HRHHID
1 HETENURE
2 HETELHHD
3 HEFAMINC
4 HRNUMHOU
5 GEREG
6 GEDIV
7 GTINDVPC
8 PRTAGE
9 PEMARITL
10 PESEX
11 PEAFEVER
12 PEEDUCA
13 PTDTRACE
14 PUCHINHH
15 PEHSPNON
16 PENATVTY
17 PRCITSHP
18 PUWK
19 PUBUS1
20 PUBUS2OT
21 PUDIS
22 PERET1
23 PRIOELG
24 PRCOW1
25 PRCOW2
26 PRCOWPG
27 PEERNPER
28 PEERNHRY
29 PUERNH1C
30 PTHR
31 PEERNHRO
32 PEERNLAB
33 PRCHLD
34 PRNMCHLD
35 race-binary


In [5]:
with open(file, 'r') as fi:
    reader = csv.reader(fi)
    headers = next(reader)
    
    white = []
    nonwhite = []
    
    for line in reader:
        if line[13] == 'white':
            print(line[13])